# Notebook for BigQuery extraction

### All data will be restricted only to **domestic, NNA and CC**

---
### Notebook configuration

In [2]:
%reload_ext cloudmagics.bigquery
%config PPMagics.autolimit=0
%config PPMagics.domain="ccg24-hrzana-gfr-interactive"

In [3]:
%load_ext autoreload
%autoreload 2

import re
import os
import sys
import warnings
import numpy as np
import pandas as pd

#user defined imports
sys.path.insert(0, os.path.dirname(os.path.realpath("__file__")) + "/../src")
import process as pr

pd.set_option('display.max_rows',     1000)
pd.set_option('display.max_columns',  1000)

warnings.filterwarnings('ignore')

#read configuration file
config          = pr.load_config()
include         = list(dict.fromkeys(config.include.model + config.include.metrics))
autocancel_file = config.data.raw.dir + '/' + config.data.raw.name.auto_cancel + '.zip'
completed_file  = config.data.raw.dir + '/' + config.data.raw.name.completed   + '.zip'
all_file        = config.data.raw.dir + '/' + config.data.raw.name.all         + '.zip'

In [4]:
#drop the unnecessary columns 
def drop_columns(df):
    # Remove unecesary fields
    exclude = [i for i in df.columns 
                   if i.endswith('_1') or 
                      i.startswith('dt') or
                      i.startswith('int') or
                      i.startswith('dec') or
                      re.match('^char\d.*', i)] + \
              [i for i in config.exclude 
                   if i in df.columns and 
                      i not in include]

    return df.drop(exclude, axis=1)

# returns the merged sf21 score from tahoe
def get_sf21(data):
    subset            = data[['send_xfer_id', 'model2_xoom21_sf_domestic_score']].copy()
    subset.columns    = ['send_xfer_id', 'sf_score']
    merge             = subset.merge(sf21, on='send_xfer_id', how='left')
    merge['sf_score'] = merge['sf_score'].replace('None', np.nan)
    
    return merge['sf_score'].combine_first(merge['xm_sf_21_releasev0_model_score1'])

#save the dataframe to a csv in raw format
def save_data(file, data, drop_columns=True):
    #drop unecesary features with specific pattern in the name
    if drop_columns:
        data = drop_columns(data)
        
    #merge the sf21 score and segment from other dataset
    data['model2_xoom21_sf_domestic_score'] = get_sf21(data)
    
    #save autocancel data to csv
    data.to_csv(file, index=False, compression='xz')

---

### 1. Retriving data for auto-canceled population

Query the BQ database

In [5]:
data = %ppbq \
SELECT  IF(uds.ctrl_is_cg = 1, uds.is_bad = 0, uds.is_fraud = false) AS target, \
        CASE WHEN uds.payment_type=4 THEN 'S0' \
             WHEN (uds.sender_trust_level=1 or uds.sender_trust_level=2 or uds.sender_trust_level=5) THEN 'S01_Trusted' \
             WHEN uds.is_first_transaction_for_sender=1 and uds.send_funding_wallet_type="XOOM" THEN 'S02_XmWall_1st' \
             WHEN uds.is_first_transaction_for_sender=1 and uds.send_funding_wallet_type="PAYPAL" and uds.login_paypal_xsell_accountage<=365 THEN 'S03_PpWall_PpYng365_Xm1st' \
             WHEN uds.is_first_transaction_for_sender=1 and uds.send_funding_wallet_type="PAYPAL" and uds.login_paypal_xsell_accountage>365 THEN "S04_PpWall_PpRU365_Xm1st" \
             WHEN uds.accountage=0 and uds.is_first_transaction_for_sender = 0 THEN 'S05_XmYng0_Fq' \
             WHEN uds.accountage>0 and uds.accountage<=90 and (uds.sender_limit_transaction_count_7_day>2 or uds.sender_limit_amount_total_7_day>=5000) THEN 'S06_XmYng1_90_HFq2' \
             WHEN uds.accountage>0 and uds.accountage<=90 and (uds.sender_limit_transaction_count_7_day>1 or uds.sender_limit_amount_total_7_day>=3000) THEN 'S07_XmYng0_90_MFq1' \
             WHEN uds.accountage>0 and uds.accountage<=90 THEN 'S08_XmYng0_90_LFq1' \
             WHEN uds.accountage>90 and uds.days_since_payment_source_added_to_uid<=7 THEN "S09_XmRU90_PsYng7" \
             WHEN uds.accountage>90 and (uds.sender_limit_transaction_count_7_day>2 or uds.sender_limit_amount_total_7_day/uds.sender_limit_amount_total_180_day>0.5) THEN "S10_XmRU90_HFq2_HEG" \
             WHEN uds.accountage>90 and (uds.sender_limit_transaction_count_7_day<=2 and uds.sender_limit_amount_total_7_day/uds.sender_limit_amount_total_180_day<=0.5) THEN "S11_XmRU90_LFq2_LEG" \
        ELSE "S0" end as xss_xoom_sf_segment2, \
        uds.*, ss.* \
    FROM pypl-bods.pp_risk_rap_xoom_managed_views.xoom_unified_dataset uds \
    INNER JOIN pypl-bods.pp_risk_rap_xoom_managed_views.xoom_signals ss \
    USING (send_xfer_id) \
        WHERE uds.created BETWEEN "2022-01-15" AND "2022-05-31" \
        AND uds.year = 2022 \
        AND uds.is_domestic = 1 \
        AND uds.is_first_transaction_for_sender = 1 \
        AND uds.payment_type = 1 \
        AND (uds.fraud_decision = "Auto Cancel" \
             OR (uds.xss_xoom_native_joint_decision = "JT-Cancel" AND uds.ctrl_is_cg = 1))

autocancel = data.DataFrame()
autocancel.shape

Establishing new BigQuery Client


HTML(value='')

job id =  ii1mn4dHja10VWdEeAR9cHmJtFQ
Time executed =  -63791965275.443  seconds
..........................

HTML(value='')

(25374, 913)

---

### 2. Retriving data for completed transactions population (latest_status = 700)

Query the BQ database

In [6]:
data = %ppbq \
SELECT  IF(uds.ctrl_is_cg = 1, uds.is_bad = 0, uds.is_fraud = false) AS target, \
        CASE WHEN uds.payment_type=4 THEN 'S0' \
             WHEN (uds.sender_trust_level=1 or uds.sender_trust_level=2 or uds.sender_trust_level=5) THEN 'S01_Trusted' \
             WHEN uds.is_first_transaction_for_sender=1 and uds.send_funding_wallet_type="XOOM" THEN 'S02_XmWall_1st' \
             WHEN uds.is_first_transaction_for_sender=1 and uds.send_funding_wallet_type="PAYPAL" and uds.login_paypal_xsell_accountage<=365 THEN 'S03_PpWall_PpYng365_Xm1st' \
             WHEN uds.is_first_transaction_for_sender=1 and uds.send_funding_wallet_type="PAYPAL" and uds.login_paypal_xsell_accountage>365 THEN "S04_PpWall_PpRU365_Xm1st" \
             WHEN uds.accountage=0 and uds.is_first_transaction_for_sender = 0 THEN 'S05_XmYng0_Fq' \
             WHEN uds.accountage>0 and uds.accountage<=90 and (uds.sender_limit_transaction_count_7_day>2 or uds.sender_limit_amount_total_7_day>=5000) THEN 'S06_XmYng1_90_HFq2' \
             WHEN uds.accountage>0 and uds.accountage<=90 and (uds.sender_limit_transaction_count_7_day>1 or uds.sender_limit_amount_total_7_day>=3000) THEN 'S07_XmYng0_90_MFq1' \
             WHEN uds.accountage>0 and uds.accountage<=90 THEN 'S08_XmYng0_90_LFq1' \
             WHEN uds.accountage>90 and uds.days_since_payment_source_added_to_uid<=7 THEN "S09_XmRU90_PsYng7" \
             WHEN uds.accountage>90 and (uds.sender_limit_transaction_count_7_day>2 or uds.sender_limit_amount_total_7_day/uds.sender_limit_amount_total_180_day>0.5) THEN "S10_XmRU90_HFq2_HEG" \
             WHEN uds.accountage>90 and (uds.sender_limit_transaction_count_7_day<=2 and uds.sender_limit_amount_total_7_day/uds.sender_limit_amount_total_180_day<=0.5) THEN "S11_XmRU90_LFq2_LEG" \
        ELSE "S0" end as xss_xoom_sf_segment2, \
        uds.*, ss.* \
    FROM pypl-bods.pp_risk_rap_xoom_managed_views.xoom_unified_dataset uds \
    INNER JOIN pypl-bods.pp_risk_rap_xoom_managed_views.xoom_signals ss \
    USING (send_xfer_id) \
        WHERE uds.created BETWEEN "2022-01-15" AND "2022-05-31" \
        AND uds.year = 2022 \
        AND uds.is_domestic = 1 \
        AND uds.is_first_transaction_for_sender = 1 \
        AND uds.payment_type = 1 \
        AND (uds.latest_status = 700 OR (uds.fraud_agent_decision='Cancel' AND uds.is_fraud=True))

completed = data.DataFrame()
completed.shape

HTML(value='')

job id =  GcPKRJL8CDZ21g6gwRLDLjqYAdD
Time executed =  -63791965698.328  seconds
......................................................................................

HTML(value='')

(85392, 913)

---

### 3. Retriving data for all transactions population (domestic, first trans for sender and credit card)

Query the BQ database

In [7]:
data = %ppbq \
SELECT  IF(uds.ctrl_is_cg = 1, uds.is_bad = 0, uds.is_fraud = false) AS target, \
        CASE WHEN uds.payment_type=4 THEN 'S0' \
             WHEN (uds.sender_trust_level=1 or uds.sender_trust_level=2 or uds.sender_trust_level=5) THEN 'S01_Trusted' \
             WHEN uds.is_first_transaction_for_sender=1 and uds.send_funding_wallet_type="XOOM" THEN 'S02_XmWall_1st' \
             WHEN uds.is_first_transaction_for_sender=1 and uds.send_funding_wallet_type="PAYPAL" and uds.login_paypal_xsell_accountage<=365 THEN 'S03_PpWall_PpYng365_Xm1st' \
             WHEN uds.is_first_transaction_for_sender=1 and uds.send_funding_wallet_type="PAYPAL" and uds.login_paypal_xsell_accountage>365 THEN "S04_PpWall_PpRU365_Xm1st" \
             WHEN uds.accountage=0 and uds.is_first_transaction_for_sender = 0 THEN 'S05_XmYng0_Fq' \
             WHEN uds.accountage>0 and uds.accountage<=90 and (uds.sender_limit_transaction_count_7_day>2 or uds.sender_limit_amount_total_7_day>=5000) THEN 'S06_XmYng1_90_HFq2' \
             WHEN uds.accountage>0 and uds.accountage<=90 and (uds.sender_limit_transaction_count_7_day>1 or uds.sender_limit_amount_total_7_day>=3000) THEN 'S07_XmYng0_90_MFq1' \
             WHEN uds.accountage>0 and uds.accountage<=90 THEN 'S08_XmYng0_90_LFq1' \
             WHEN uds.accountage>90 and uds.days_since_payment_source_added_to_uid<=7 THEN "S09_XmRU90_PsYng7" \
             WHEN uds.accountage>90 and (uds.sender_limit_transaction_count_7_day>2 or uds.sender_limit_amount_total_7_day/uds.sender_limit_amount_total_180_day>0.5) THEN "S10_XmRU90_HFq2_HEG" \
             WHEN uds.accountage>90 and (uds.sender_limit_transaction_count_7_day<=2 and uds.sender_limit_amount_total_7_day/uds.sender_limit_amount_total_180_day<=0.5) THEN "S11_XmRU90_LFq2_LEG" \
        ELSE "S0" end as xss_xoom_sf_segment2, \
        uds.*, ss.* \
    FROM pypl-bods.pp_risk_rap_xoom_managed_views.xoom_unified_dataset uds \
    INNER JOIN pypl-bods.pp_risk_rap_xoom_managed_views.xoom_signals ss \
    USING (send_xfer_id) \
        WHERE uds.created BETWEEN "2022-01-15" AND "2022-05-31" \
        AND uds.year = 2022 \
        AND uds.is_domestic = 1 \
        AND uds.is_first_transaction_for_sender = 1 \
        AND uds.payment_type = 1

_all = data.DataFrame()
_all.shape

HTML(value='')

job id =  Y7BteRA4nE8IC50jP8VyXfeSpJh
Time executed =  -63791966881.41  seconds
...........................................................................................................................

HTML(value='')

(122915, 913)

---

### 4. Retriving data SF_21 score Jan-May 2022

Query the Hive database

In [8]:
data = %hive_tahoe \
SELECT  send_xfer_id, \
        xm_sf_21_releasev0_model_score1 \
    FROM xoom_SF21_driverset0618 sf21 \
        WHERE sf21.year = 2022 \

sf21 = data.DataFrame()
sf21.shape

HTML(value='')

HTML(value='')

HTML(value='')

(1295209, 2)

---

### 5. Saving data to the project

Removing columns that are not going to be used and export data to zipped csv

In [9]:
save_data(all_file, _all, drop_columns=False)
save_data(completed_file, completed, drop_columns=False)
save_data(autocancel_file, autocancel, drop_columns=False)

Cleaning the data

In [10]:
#import gc

#del [[data]]
#gc.collect()